# Дообучаем модель T5-base (но можно использовать другие seq2seq модели) для упрощения текстов

Устанавливаем библиотеку easse для оценки моделей. Иногда при установке возникает ошибка - в таком случае можно просто перезапустить среду.

In [1]:
# !pip install --upgrade setuptools
# !pip install ez_setup

# !git clone https://github.com/feralvam/easse
# !git clone https://github.com/Andoree/sent_simplification.git
# !cp sent_simplification/sari.py /content/easse/easse

# !pip install -r text_simplifier/experiments/requirements.txt
# !pip install -r text_simplifier/seq2seq/requirements.txt

# !git clone https://github.com/dialogue-evaluation/RuSimpleSentEval
# !git clone https://github.com/Digital-Pushkin-Lab/RuAdapt

Собираем корпус из двух источников.

In [2]:
import pandas as pd
from data import SOURCE_COLUMN_NAME, TARGET_COLUMN_NAME, filter_ru_adapt, info
from paths import RU_ADAPT_PATHS, RUSIMPLESENTEVAL_PATH, TEST_DATA_PATH

senteval = pd.read_csv(RUSIMPLESENTEVAL_PATH)
ru_adapt_dfs = []
ru_adapt_dfs.append(filter_ru_adapt(RU_ADAPT_PATHS[0], min_sim = 0.75, min_lev = 20, max_elong_rate = 0.1))
for path in RU_ADAPT_PATHS[1:]:
  ru_adapt_dfs.append(filter_ru_adapt(path, min_sim = 0.5, min_lev = 5, max_elong_rate = 0.1))

data = pd.concat([senteval]+ru_adapt_dfs)[[SOURCE_COLUMN_NAME, TARGET_COLUMN_NAME]]  
info([RUSIMPLESENTEVAL_PATH] + RU_ADAPT_PATHS, [senteval] + ru_adapt_dfs)

Name: RuSimpleSentEval\dev_sents.csv  Size: 3406
Name: RuAdapt\Adapted_literature\zlatoust_sentence_aligned_with_CATS.csv  Size: 923
Name: RuAdapt\Encyclopedic\lsslovar_B_to_A_sent.csv  Size: 294
Name: RuAdapt\Encyclopedic\lsslovar_C_to_A_sent.csv  Size: 286
Name: RuAdapt\Encyclopedic\lsslovar_C_to_B_sent.csv  Size: 1202
Name: RuAdapt\Fairytales\df_fairytales_sent.csv  Size: 65


In [3]:
test_data = pd.read_csv(TEST_DATA_PATH)
test_data = test_data.rename(columns = {
    SOURCE_COLUMN_NAME: SOURCE_COLUMN_NAME, 
    TARGET_COLUMN_NAME: TARGET_COLUMN_NAME})
test_data = test_data[[SOURCE_COLUMN_NAME,TARGET_COLUMN_NAME]]
info([TEST_DATA_PATH], [test_data])

Name: RuSimpleSentEval\public_test_sents.csv  Size: 3398


Приводим данные в формат, подходящий для подсчета метрик.

In [4]:
display(data[SOURCE_COLUMN_NAME].iloc[0])
display(data[TARGET_COLUMN_NAME].iloc[0])
print('----')
display(test_data[SOURCE_COLUMN_NAME].iloc[0])
display(test_data[TARGET_COLUMN_NAME].iloc[0])

'14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.'

'14 декабря 1944 года рабочий посёлок Ички переименован в Советский.'

----


'14 апреля 2003 году архиепископом Новосибирским и Бердским Тихоном пострижен в монашество с наречением имени Феодор в честь праведного Феодора Томского.'

'Был пострижен в монашество и получил имя Фёдор в честь Федора Томского.'

In [5]:
from data import prepare_data_for_eval

orig, refs = prepare_data_for_eval(data)
test_orig, test_refs = prepare_data_for_eval(test_data)
print(f'Unique train sents: {len(orig)}')
print(f'Unique test sents: {len(test_orig)}')


Unique train sents: 3454
Unique test sents: 1000


In [6]:
print(orig[0])
print(refs[0][0])

14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.
14 декабря 1944 года рабочий посёлок Ички переименован в Советский.


Загружаем модель и токенизатор.

In [7]:
from paths import get_checkpoints_path
from model import get_model, get_tokenizer

MODEL_ID = 'rut5-base'
DATASET = 'pls'

checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)
print(checkpoints_path)

Loading pretrained model sberbank-ai/ruT5-base


In [9]:
from utils import sent_length_info
sent_length_info(data[SOURCE_COLUMN_NAME], tokenizer)

Average text length: 32.284650259067355
Median text length: 29.0
Max text length: 243


Сплитим данные, токенизируем тексты, делим их на батчи.

In [18]:
from dataset import Dataset
from torch.utils.data import DataLoader
from utils import RANDOM_STATE, BATCH_SIZE
from sklearn.model_selection import train_test_split
from paths import get_checkpoints_path
from model import get_model, get_tokenizer
from inference import batch_inference, example
from evaluation import compute_corpus_metrics
from utils import stringify_dict

ml=50
data_multitask = data.reset_index()
train_data, val_data = train_test_split(data_multitask, test_size=0.2, random_state = RANDOM_STATE)

train_dataset = Dataset(train_data, tokenizer, max_length = ml)
val_dataset = Dataset(val_data, tokenizer, max_length = ml)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

print(f'Batch size is {BATCH_SIZE}')
print(f'Train data: {len(train_dataset)} sentences')
print(f'Val data: {len(val_dataset)} sentences')

Batch size is 32
Train data: 4940 sentences
Val data: 1236 sentences


In [42]:
len(train_dataset)
# len(train_loader)

4940

In [ ]:
[{'':"","":""...}]

In [39]:
for  i in train_loader:
    print(i)
    break

TypeError: 'DataLoader' object is not subscriptable

In [11]:
print(data_multitask['INPUT:source'][0])
print(data_multitask['OUTPUT:output'][0])

14 декабря 1944 года рабочий посёлок Ички был переименован в рабочий посёлок Советский, после чего поселковый совет стал называться Советским.
14 декабря 1944 года рабочий посёлок Ички переименован в Советский.


#### Настраиваем логирование. Можно выбрать другие пути для сохранения.

In [14]:
from utils import set_logging, cleanup

logger = set_logging()

#### Обучаем модель!

In [16]:
from trainer import train

train(model, tokenizer,
      train_loader, val_loader,
      model_id = MODEL_ID, dataset_name = DATASET,
      checkpoints_path = checkpoints_path,
      logger = logger,
      orig = orig, refs = refs,
      max_epochs = 5,
      cleanup_step = 1, report_epoch = 5, save_epoch = 5,
      lr = 3e-4)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bert\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!



Epoch 1, step 154: train loss: 1.8634  val loss: 6.9560
SOURCE: После крещения верующие постоянно носят крест на груди (нательный или наперсный крест – от старославянского перси – грудь).
TARGET: Верующие постоянно носят крест на груди (нательный или наперсный крест – от старославянского перси – грудь).
PREDICTION: После крещения верующие носят крест на груди.



Epoch 2, step 154: train loss: 1.3953  val loss: 7.2499
SOURCE: Стихотворение вызвало неудовольствие властей, и поэт был выслан на Кавказ в действующую армию.
TARGET: Стихотворение вызвало неудовольствие властей, и поэт был отправлен  на Кавказ в действующую армию.
PREDICTION: Стихи поэта вызвали неудовольствие властей, и поэт был выслан на Кавказ.



Epoch 3, step 154: train loss: 1.1251  val loss: 7.1528
SOURCE: В годы Великой Отечественной войны в город были эвакуированы из Москвы правительственные учреждения, посольства и представительства иностранных государств, промышленные предприятия, Большой театр, «Мосфильм»; было налажено производство знаменитых самолетов-штурмовиков Ил-2, Ил-10 (см.
TARGET: В годы Великой Отечественной войны в город были эвакуированы из Москвы правительственные учреждения, посольства и представительства иностранных государств, промышленные предприятия.
PREDICTION: В годы Великой Отечественной войны в город эвакуированы правительственные учреждения, правительственные учреждения, правительственные учреждения, посольства и представительства иностранных государств, промышленные предприятия, Большой театр, «Мосфильм»; были налажено производство знаменитых самолетов-штурмовиков Ил-2, Ил-2, «Мосфильм».



Epoch 4, step 154: train loss: 0.9316  val loss: 7.7338
SOURCE: Язгуля́мский язы́к (самоназвание — Yuzdami zevég) — один из памирских языков, распространённый главным образом в долине реки Язгулям (западный Памир).
TARGET: Язгулямский язык - один из языков, на котором в основном говорят в долине реки Язгулям на западном Памире. 
PREDICTION: Язгулямский язы-к (самоназвание — uzdag) — это распространённый в долине реки Язгулям (западный Памир).



Epoch 5, step 154: train loss: 0.7901  val loss: 8.0258
SOURCE: В них он предстает как умный, хитрый и жестокий соперник своих политических оппонентов.
TARGET: В них он – умный, хитрый и жестокий соперник своих политических оппонентов.
PREDICTION: В них он – как умный и жестокий соперник своих политических оппонентов.
Model saved to checkpoints\rut5-base-complication.pt
Computing sari and bleu


In [23]:
from utils import reload, cleanup
cleanup()
reload('inference')
reload('trainer')

<module 'inference' from 'c:\\Users\\bert\\test\\textifyZen\\seq2seq\\inference.py'> reloaded successfully
<module 'trainer' from 'c:\\Users\\bert\\test\\textifyZen\\seq2seq\\trainer.py'> reloaded successfully


# Evaluation

In [24]:
sample_input, sample_output = test_data[SOURCE_COLUMN_NAME][0], test_data[TARGET_COLUMN_NAME][0]

In [25]:
text='Война Советского Союза против фашистской Германии и её союзников (Венгрии, Италии, Румынии, Словакии, Хорватии, Финляндии, Японии); составная часть Второй мировой войны 1939-1945 гг.'
print(f'PREDICTION: {example(text, sample_output, model, tokenizer)}')

SOURCE: Война Советского Союза против фашистской Германии и её союзников (Венгрии, Италии, Румынии, Словакии, Хорватии, Финляндии, Японии); составная часть Второй мировой войны 1939-1945 гг.
TARGET: Был пострижен в монашество и получил имя Фёдор в честь Федора Томского.
PREDICTION: Война Советского Союза против фашистской Германии (Венгрии, Италии, Румынии, Словакии, Финляндии, Японии) – составная часть Второй мировой войны 1939-1945 гг.


#### Without simplification

In [26]:
simplification_func = lambda texts: texts
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bleu:10.813 sari:1.42 fkgl:-0.962 bertscore:0.54


#### Gold Standard simplification

In [27]:
simplification_func = lambda texts: [r.iloc[0] for r in test_refs]
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bleu:100.0 sari:33.908 fkgl:16.421 bertscore:1.0


### Push-им в HF

In [22]:
from huggingface_hub import notebook_login
notebook_login()
tokenizer.push_to_hub('r1char9/ruT5-base-pls')
model.push_to_hub('r1char9/ruT5-base-pls')

### Внизу приведены другие модели для обучения !

##Ru-T5 Pretrained

In [ ]:
MODEL_ID = 'rut5-base'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = False)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##Ru-T5

In [ ]:
MODEL_ID = 'rut5-base'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##Ru-T5 Senteval

In [ ]:
MODEL_ID = 'rut5-base'
DATASET = 'senteval'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##MT5

In [ ]:
MODEL_ID = 'mt5-small'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##RuBART

In [ ]:
MODEL_ID = 'bart'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##Paraphraser Pretrained

In [ ]:
MODEL_ID = 'paraphraser'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = False)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
print(f'PREDICTION: {example(" Военные власти Российской империи с большой осторожностью включали в ряды вооружённых сил население присоединённых земель, но были более склонны инкорпорировать нерусскую знать (особенно ту её часть, которая соглашалась принять православие) в состав офицерского корпуса.", sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##Paraphraser

In [ ]:
MODEL_ID = 'paraphraser'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##RuT5 Absum Pretrained

In [ ]:
MODEL_ID = 'rut5-absum'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = False)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))

##RuT5 Absum

In [ ]:
MODEL_ID = 'rut5-absum'
DATASET = 'senteval+ruadapt'
checkpoints_path = get_checkpoints_path(MODEL_ID, DATASET)
model = get_model(MODEL_ID, checkpoints_path = checkpoints_path, from_checkpoints = True)
tokenizer = get_tokenizer(MODEL_ID)

In [ ]:
print(f'PREDICTION: {example(sample_input, sample_output, model, tokenizer)}')

In [ ]:
simplification_func = lambda texts: batch_inference(texts, model = model, tokenizer = tokenizer)
metrics, quality_estimation = compute_corpus_metrics(orig = test_orig,
                                                     refs = test_refs,
                                                     simplification_func = simplification_func,
                                                     compute_quality_estimation = False,
                                                     metrics=('bleu', 'sari', 'fkgl', 'bertscore'))
print(stringify_dict(metrics))